# *SessionM Data Generation Process*

### Assumptions for Customer1.csv:

1. id 4903g34 is duplicate which is an identifier for customer in external system but there are 2 rows with the same id with different data which might be because of some data corruption happened at the source system.

    **Correction measure taken:**
    To correct this i have assigned a new alpha numeric id to user named Sandrine.This will help in identifying the records with the correct information and after joining both the data frames (customer1 and customer2) we'll be able to make correct assumptions and processing.


2. There is space in the email address of jh41922 id i.e sam2382@mailinator .com. This might be an issue of wrong entry by the user in source system.To correct the data the spaces are trimmed.


3. Attr1 is not showing any meaningful information related to customer. Hence this will be dropped from dataframe.


4. Engagement is not present in SessionM API documentation and is not adding any insights to the data, hence will be dropped from the dataframe.


### Assumptions for Customer2.csv:
1) By looking at the data '0' under sex column means Male (m) and '1' means female (f).

2) By looking at the data and sessionM api documentation 'tier' means the plans opted by customer.In this csv all the customers are opted-in for plans hence the value would be True under 'opted_in' column and details will be kept under custom_attr1.

3) By looking at the data 'lastcontact' shows last time the customer was contacted or used any service.Since this field is not mentioned in the API documentation hence will be placed under custom_attr1.Keeping this field because it can be used if someone whats to get insights of data for the customers which are active recently.

   **Correction measure taken:**
    The format of last contacted date is inconsistent. Hence converting all in YYYY-MM-DD format.

4) By looking at the data under 'pets' column , 0 means no pets own by a customer and 1 means pets own by a customer. As per SessionM API documentation, there is no column related to 'pets' information and and is not adding any insights to the data,  hence dropping the column.

5) By looking at the data 'attr1' shows the vehicle owned by a customer. Since this field is not mentioned in the API documentation and is not adding any insights to the data, hence dropping the column.

6) By looking at the data 'attr2' shows the phone number of customers. This should be placed under phone number column. Also there is one more information which can be derived from phone number i.e country. +1 is a code for USA, which can be placed under country column.


### Questions:

1) How the id 4903g34 is same for 2 records from the source. This should be a random generated number from source system. Can the same id be assigned to other user as well , or is there a concept of expiration account on the source system i.e. id of the account which is now expired can be assigned to other user?

2) How come sam2382@mailinator .com with spaces in the email id column allowed in the source system. Is there no validation on email id field on source?

3) What is the use of engagement,attr1 in customer1.csv and pets,attr1 in customer2.csv as no information is derived from these columns which can be used in other columns mentioned in API documentation?



In [263]:
import pandas as pd

# loading both the CSVs in 2 different pandas dataframe
df1=pd.read_csv('customer1.csv')
df2= pd.read_csv('customer2.csv')

In [264]:
df1

,id,first_name,last_name,email,engagement,attr1
0,4903g34,Joe,Johnson,joe.johnson@spamhole.com,49.0,red
1,48982nf,Mike,Jackson,mj@temp-mail.org,422.0,oil
2,4903g34,Sandrine,Jervais,sjervai@generator.email,4.0,linen
3,jh41922,Samantha,Holland,sam2382@mailinator .com,24.0,Grassy


In [257]:
df2

,id,sex,tier,lastcontact,pets,attr1,attr2
0,4903g34,0,platinum,1/24/2018,0,minivan,+1 569-483-2388
1,48982nf,0,red,1/12/2018,1,car,+1 555-555-1212
2,an2944a,1,gold,2/17/2018,1,pickuptruck,+1 432-867-5309
3,jh41922,1,blue,11/4/2017,0,Car,+1 716-873-9837


In [258]:
# finding duplicate id and replacing it with unique alphanumeric code to maintain consistence and to join with other data frame.
df1.loc[df1.duplicated(['id']),'id']='4903x34'
df1

,id,first_name,last_name,email,engagement,attr1
0,4903g34,Joe,Johnson,joe.johnson@spamhole.com,49.0,red
1,48982nf,Mike,Jackson,mj@temp-mail.org,422.0,oil
2,4903x34,Sandrine,Jervais,sjervai@generator.email,4.0,linen
3,jh41922,Samantha,Holland,sam2382@mailinator .com,24.0,Grassy


In [259]:
import numpy as np
new_df=pd.merge(df1, df2, on='id', how='outer')
new_df.loc[new_df.first_name=='Sandrine',['sex']]='1'
new_df.rename(index=str,
              columns={"attr2":"phone_numbers","id":"external_id","sex":"gender","tier":"custom_1","lastcontact":"custom_2"},
              inplace=True)

# tranlating 0 and 1 to Male and Female
new_df['gender'] = np.where(new_df['gender'] ==0, 'm', 'f')

# dropping the below columns which are not required:
# attr1_x from customer1.csv -- not making any sense 
# engagement from customer1.csv -- not making any sense
# pets from customer2.csv -- not making any sense
# vehcle from customer2.csv -- not making any sense
new_df.drop(['attr1_x','engagement','pets','attr1_y'],inplace=True,axis =1)

# removing spaces from email id column
new_df['email']=new_df['email'].str.replace(' ','')

# adding the below columns as per session m API documentation
# opted_in : defaults to true if no attribute value is specified hence true for all records
# external_id_type : This represents from which platform the data is received "facebook,instagram etc".NaN for no details
# locale : by looking at phone numbers since it's of USA so locale should be en-u for all records
# ip : NaN for no details
# dob : NaN for no details
# address : NaN for no details
# city : NaN for no details
# state : NaN for no details
# zip : NaN for no details
# country : by looking at phone numbers since it's of USA so country should be USA for all records
# referral : NaN for no details -- this can be generated while processing data using NAME-XXXXXX but kept blank as not received from source
# phone_type : NaN for no details


new_df['opted_in'],new_df['external_id_type'],new_df['locale'],new_df['ip'],new_df['dob'],new_df['address'],new_df['city'],new_df['state'],new_df['zip'],new_df['country'],new_df['referral'],new_df['phone_type']=[True,np.nan,'en-u',np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,'USA',np.nan,np.nan]

# correcting the date format present in custom_2 column
new_df['custom_2'] = pd.to_datetime(new_df['custom_2']).dt.strftime('%Y-%m-%d')


# removing NaT from custom_2 field
new_df['custom_2'] =  new_df['custom_2'].astype(str)
new_df['custom_2'] = new_df['custom_2'].apply(lambda val : np.nan if val=="NaT" else val)

new_df.head()

,external_id,first_name,last_name,email,gender,custom_1,custom_2,phone_numbers,opted_in,external_id_type,locale,ip,dob,address,city,state,zip,country,referral,phone_type
0,4903g34,Joe,Johnson,joe.johnson@spamhole.com,m,platinum,2018-01-24,+1 569-483-2388,True,NaN,en-u,NaN,NaN,NaN,NaN,NaN,NaN,USA,NaN,NaN
1,48982nf,Mike,Jackson,mj@temp-mail.org,m,red,2018-01-12,+1 555-555-1212,True,NaN,en-u,NaN,NaN,NaN,NaN,NaN,NaN,USA,NaN,NaN
2,4903x34,Sandrine,Jervais,sjervai@generator.email,f,NaN,NaN,NaN,True,NaN,en-u,NaN,NaN,NaN,NaN,NaN,NaN,USA,NaN,NaN
3,jh41922,Samantha,Holland,sam2382@mailinator.com,f,blue,2017-11-04,+1 716-873-9837,True,NaN,en-u,NaN,NaN,NaN,NaN,NaN,NaN,USA,NaN,NaN
4,an2944a,NaN,NaN,NaN,f,gold,2018-02-17,+1 432-867-5309,True,NaN,en-u,NaN,NaN,NaN,NaN,NaN,NaN,USA,NaN,NaN


In [260]:
# Arranging the columns in the order mentioned in API documentation
new_df=new_df[['external_id','opted_in','external_id_type','email','locale','ip','dob','address','city','state','zip','country',
         'gender','first_name','last_name','referral','phone_numbers','phone_type','custom_1','custom_2']]


# final data frame
print(45 *'*','Final dataframe',45*'*')

new_df


********************************************* Final dataframe *********************************************


,external_id,opted_in,external_id_type,email,locale,ip,dob,address,city,state,zip,country,gender,first_name,last_name,referral,phone_numbers,phone_type,custom_1,custom_2
0,4903g34,True,NaN,joe.johnson@spamhole.com,en-u,NaN,NaN,NaN,NaN,NaN,NaN,USA,m,Joe,Johnson,NaN,+1 569-483-2388,NaN,platinum,2018-01-24
1,48982nf,True,NaN,mj@temp-mail.org,en-u,NaN,NaN,NaN,NaN,NaN,NaN,USA,m,Mike,Jackson,NaN,+1 555-555-1212,NaN,red,2018-01-12
2,4903x34,True,NaN,sjervai@generator.email,en-u,NaN,NaN,NaN,NaN,NaN,NaN,USA,f,Sandrine,Jervais,NaN,NaN,NaN,NaN,NaN
3,jh41922,True,NaN,sam2382@mailinator.com,en-u,NaN,NaN,NaN,NaN,NaN,NaN,USA,f,Samantha,Holland,NaN,+1 716-873-9837,NaN,blue,2017-11-04
4,an2944a,True,NaN,NaN,en-u,NaN,NaN,NaN,NaN,NaN,NaN,USA,f,NaN,NaN,NaN,+1 432-867-5309,NaN,gold,2018-02-17


In [262]:
# Exporting data in CSV
new_df.to_csv('Combined_Customer_data.csv')